In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

In [2]:
metadata_recovery_date = '2020-07-31'

In [4]:
metadata_table_path = '/mnt/customer360-blob-output/C360/UTILITIES/metadata_table/'
metadata_table_backup_path = '/mnt/customer360-blob-output/C360/UTILITIES/util_metadata_backup/'

In [5]:
def metadata_recovery(metadata_table_path, metadata_table_backup_path, metadata_recovery_date):
  def get_new_sparkSession():
    spark = SparkSession.builder.appName("project_customer_360_metadata_recovery").getOrCreate()
    spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")
    spark.conf.set("spark.sql.parquet.binaryAsString", "true")
    spark.conf.set("spark.sql.shuffle.partitions", 200)
    spark.conf.set("spark.sql.files.maxPartitionBytes", 1024*1024*256)
    return spark
  spark = get_new_sparkSession()
  spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")
  
  mtdt_tbl = spark.read.option("mergeSchema","true").parquet(metadata_table_backup_path)
  mtdt_tbl = mtdt_tbl.filter(f.col("metadata_table_backup_date") == f.to_date(f.lit(metadata_recovery_date), "yyyy-MM-dd")).drop("metadata_table_backup_date")
  
  mtdt_tbl.write.partitionBy("table_name").format("parquet").mode("overwrite").save(metadata_table_path)

In [6]:
metadata_recovery(metadata_table_path, metadata_table_backup_path, metadata_recovery_date)